In [40]:
#IMPORTS
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV #splits up your test set in eqally sized parts, uses one part as test data and the rest as training data. So it optimizes as many classifiers as parts you split your data into.


In [42]:
#LOAD DATA

df = pd.read_csv('USER-FEATURE-DATA.csv')
current_row=0
df_sample = pd.DataFrame([],columns=df.columns)
for i in np.unique(df['user1']):
      new_sample=df[df['user1']==i].sample(n=5)
      df_sample=pd.concat([df_sample,new_sample],axis=0)

df_sample = df_sample.reset_index(drop=True)

y = df_sample.iloc[:, 0]
X = df_sample.iloc[:, 1:]
print(y[5]) 

user19


In [45]:
#TRAINING

kf = KFold(n_splits=5)

ATTRIBUTES = len(X.columns)-1
print('n attrb =', ATTRIBUTES)
USERS = y.nunique()
print('n users =', USERS)
print(USERS)
neurons = (ATTRIBUTES + USERS) // 2 
clf = MLPClassifier(hidden_layer_sizes=(neurons,), solver='lbfgs', momentum=0.2, learning_rate_init=0.3, random_state=0)
clf.n_layers_ = 3
clf.max_iter = 1000
#got rid of:
#hidden_layer_sizes=(1,) (scores all zeroes)
#  l
#
# look into RandomSearchCV
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd'],
            'hidden_layer_sizes': [
              (neurons-10,), (neurons-5,), (neurons,), (neurons+5,), (neurons+10,)
             ],
             'momentum': [0.1,0.2,0.3],
             'learning_rate_init':[0.2,0.3,0.4]
        }
       ]

GRID_SEARCH_TEST = False 

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    if GRID_SEARCH_TEST:
        clf = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
        clf.fit(X_train, y_train)
        print('params:', clf.best_params_)
    else:
      clf.fit(X_train, y_train)
      print('SCORE:', clf.score(X_test, y_test))
    """ 
    EXTRA INFO:
    # 

      print('current loss computed with the loss function: ',clf.loss_)
      #print('coefs: ', clf.coefs_)
      print('intercepts: ',clf.intercepts_)
      print(' number of iterations the solver: ', clf.n_iter_)
      print('num of layers: ', clf.n_layers_)
      print('Num of o/p: ', clf.n_outputs_) 
      
      """



n attrb = 101
n users = 20
SCORE: 0.0


c:\Users\Jagrit Rai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


SCORE: 0.0
SCORE: 0.0
SCORE: 0.0
SCORE: 0.0
